# Генерация парафразов для тестирования чат-бота

Используем предобученные модели Hugging Face для генерации и валидации парафразов.

## Содержание
1. [Импорты и настройка](#1-импорты-и-настройка)
2. [Утилиты для определения типа предложения](#2-утилиты)
3. [Генератор парафразов](#3-генератор-парафразов)
4. [Валидатор парафразов](#4-валидатор-парафразов)
5. [Основной пайплайн](#5-основной-пайплайн)
6. [Демонстрация работы](#6-демонстрация)

## 1. Импорты и настройка

In [1]:
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")

c:\Users\tizza\Projects\nlp_homework\task3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Используемое устройство: cpu


## 2. Утилиты

Функции для определения типа предложения (вопрос, команда, утверждение) и проверки сохранения типа.

In [2]:
def detect_sentence_type(text: str) -> str:
    """Определение типа предложения: question, command, statement."""
    text = text.strip()
    if text.endswith("?"):
        return "question"
        
    command_starters = [
        "set", "turn", "please", "remind", "call", "send", "open", "close",
        "play", "stop", "start", "show", "tell", "find", "search", "get",
        "make", "create", "delete", "remove", "add", "enable", "disable",
        "switch", "change", "update", "check", "let", "help", "give"
    ]
    first_word = text.split()[0].lower().rstrip(",")
    if first_word in command_starters:
        return "command"
    return "statement"


def preserve_sentence_type(original: str, paraphrase: str) -> bool:
    """Проверка, что тип предложения сохранён."""
    orig_type = detect_sentence_type(original)
    para_type = detect_sentence_type(paraphrase)
    
    if orig_type == "question":
        return para_type == "question"
    
    if orig_type == "command":
        return para_type != "question"
    
    return True

### Тестирование утилит

In [3]:
test_sentences = [
    "What's the weather like today?",
    "Please turn on the lights.",
    "Set an alarm for 7 AM.",
    "The weather is nice today.",
    "After your workout, remember to drink water."
]

print("Определение типа предложений:")
print("-" * 60)
for sentence in test_sentences:
    sent_type = detect_sentence_type(sentence)
    print(f"[{sent_type:10}] {sentence}")

Определение типа предложений:
------------------------------------------------------------
[question  ] What's the weather like today?
[command   ] Please turn on the lights.
[command   ] Set an alarm for 7 AM.
[statement ] The weather is nice today.
[statement ] After your workout, remember to drink water.


## 3. Генератор парафразов

Класс для генерации парафразов на основе T5-подобных моделей. Поддерживает различные стратегии генерации:
- **beam** - beam search
- **diverse_beam** - diverse beam search с sampling
- **sampling** - top-k sampling
- **nucleus** - nucleus (top-p) sampling

In [4]:
class ParaphraseGenerator:
    """
    Генератор парафразов на основе T5-подобных моделей.
    Поддерживает различные стратегии генерации: beam search, sampling, nucleus sampling.
    """
    
    def __init__(
        self,
        model_name: str = "humarin/chatgpt_paraphraser_on_T5_base",
        device: Optional[str] = None
    ):
        """
        Инициализация генератора.
        
        Args:
            model_name: Название модели из Hugging Face Hub
            device: Устройство для вычислений ('cuda', 'cpu' или None для автовыбора)
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Загрузка модели генерации: {model_name}")
        print(f"Устройство: {self.device}")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
        self.model.eval()
        
        self.model_name = model_name
        print("Модель загружена успешно!")
        
    def generate(
        self,
        text: str,
        num_paraphrases: int = 10,
        strategy: str = "nucleus",
        max_length: int = 128,
        temperature: float = 1.0,
        top_p: float = 0.9,
        top_k: int = 50
    ) -> List[str]:
        """
        Генерация парафразов для входного текста.
        
        Args:
            text: Исходный текст для парафразирования
            num_paraphrases: Количество парафразов для генерации
            strategy: Стратегия генерации ('beam', 'diverse_beam', 'sampling', 'nucleus')
            max_length: Максимальная длина генерируемого текста
            temperature: Температура для sampling (выше = более разнообразно)
            top_p: Параметр для nucleus sampling
            top_k: Параметр для top-k sampling
            
        Returns:
            Список сгенерированных парафразов
        """
        sentence_type = detect_sentence_type(text)
        
        if "paraphras" in self.model_name.lower():
            input_text = f"paraphrase: {text}"
        elif "flan" in self.model_name.lower():
            if sentence_type == "question":
                input_text = f"Paraphrase this question while keeping it as a question: {text}"
            elif sentence_type == "command":
                input_text = f"Paraphrase this command while keeping it as a command or request: {text}"
            else:
                input_text = f"Paraphrase the following sentence: {text}"
        else:
            input_text = f"Paraphrase the following sentence: {text}"
        
        inputs = self.tokenizer(
            input_text,
            return_tensors="pt",
            max_length=max_length,
            truncation=True,
            padding=True
        ).to(self.device)
        
        gen_kwargs = {
            "max_length": max_length,
            "early_stopping": True,
            "pad_token_id": self.tokenizer.pad_token_id,
        }
        
        if strategy == "beam":
            gen_kwargs.update({
                "num_beams": num_paraphrases,
                "num_return_sequences": num_paraphrases,
            })
        elif strategy == "diverse_beam":
            gen_kwargs.update({
                "num_beams": num_paraphrases,
                "num_return_sequences": num_paraphrases,
                "do_sample": True,
                "temperature": 1.2,
            })
        elif strategy == "sampling":
            gen_kwargs.update({
                "do_sample": True,
                "num_return_sequences": num_paraphrases,
                "temperature": temperature,
                "top_k": top_k,
            })
        elif strategy == "nucleus":
            gen_kwargs.update({
                "do_sample": True,
                "num_return_sequences": num_paraphrases,
                "temperature": temperature,
                "top_p": top_p,
            })
        else:
            raise ValueError(f"Неизвестная стратегия: {strategy}")
        
        with torch.no_grad():
            outputs = self.model.generate(**inputs, **gen_kwargs)
        
        paraphrases = []
        for output in outputs:
            decoded = self.tokenizer.decode(output, skip_special_tokens=True)
            if decoded.strip() and decoded.strip().lower() != text.strip().lower():
                paraphrases.append(decoded.strip())
        
        seen = set()
        unique_paraphrases = []
        for p in paraphrases:
            p_lower = p.lower()
            if p_lower not in seen:
                seen.add(p_lower)
                unique_paraphrases.append(p)
        
        return unique_paraphrases

### Инициализация генератора

In [5]:
generator = ParaphraseGenerator()

Загрузка модели генерации: humarin/chatgpt_paraphraser_on_T5_base
Устройство: cpu
Модель загружена успешно!


### Тестирование генератора

In [6]:
test_text = "What's the weather like today?"
print(f"Исходный текст: {test_text}")
print(f"Тип: {detect_sentence_type(test_text)}")
print("\nСгенерированные парафразы:")
print("-" * 50)

paraphrases = generator.generate(test_text, num_paraphrases=5, strategy="nucleus")
for i, p in enumerate(paraphrases, 1):
    print(f"{i}. {p}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Исходный текст: What's the weather like today?
Тип: question

Сгенерированные парафразы:
--------------------------------------------------
1. What's the weather forecast for today?
2. Is there any news on the weather today?
3. What's the current weather like?


### Сравнение стратегий генерации

In [7]:
test_text = "Please turn on the lights in the living room."
strategies = ["beam", "sampling", "nucleus"]

print(f"Исходный текст: {test_text}")
print("=" * 60)

for strategy in strategies:
    print(f"\nСтратегия: {strategy}")
    print("-" * 40)
    paraphrases = generator.generate(test_text, num_paraphrases=3, strategy=strategy)
    for i, p in enumerate(paraphrases, 1):
        print(f"  {i}. {p}")

Исходный текст: Please turn on the lights in the living room.

Стратегия: beam
----------------------------------------
  1. Please activate the lights fixture in the living room.
  2. It is your request that the living room be lit up.
  3. We request that you turn on the lights in the lounge.

Стратегия: sampling
----------------------------------------
  1. So, hold on; light up the living room.
  2. It would be appreciated if you could switch on the lights in the living room.
  3. Kindly enable the lights in the living room.

Стратегия: nucleus
----------------------------------------
  1. It is my responsibility to light up the living room.
  2. Please, please turn on the lights in the living room.
  3. I request that you put on the lights in the living room.


## 4. Валидатор парафразов

Класс для проверки качества парафразов с использованием:
- **Embedding similarity** - косинусное сходство эмбеддингов
- **Lexical diversity** - лексическое разнообразие (1 - Jaccard similarity)
- **NLI (Natural Language Inference)** - проверка логического следования

In [8]:
class ParaphraseValidator:
    """
    Валидатор качества парафразов.
    Использует embedding similarity и NLI для проверки семантической эквивалентности.
    """
    
    def __init__(
        self,
        embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2",
        nli_model: str = "facebook/bart-large-mnli",
        device: Optional[str] = None
    ):
        """
        Инициализация валидатора.
        
        Args:
            embedding_model: Модель для вычисления эмбеддингов
            nli_model: Модель для NLI (Natural Language Inference)
            device: Устройство для вычислений
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"Загрузка модели эмбеддингов: {embedding_model}")
        self.embedding_model = SentenceTransformer(embedding_model, device=self.device)
        
        print(f"Загрузка NLI модели: {nli_model}")
        self.nli_pipeline = pipeline(
            "zero-shot-classification",
            model=nli_model,
            device=0 if self.device == "cuda" else -1
        )
        print("Модели валидатора загружены успешно!")
        
    def compute_similarity(self, text1: str, text2: str) -> float:
        """
        Вычисление косинусного сходства между двумя текстами.
        
        Args:
            text1: Первый текст
            text2: Второй текст
            
        Returns:
            Косинусное сходство (от 0 до 1)
        """
        embeddings = self.embedding_model.encode([text1, text2])
        similarity = np.dot(embeddings[0], embeddings[1]) / (
            np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1])
        )
        return float(similarity)
    
    def compute_lexical_diversity(self, text1: str, text2: str) -> float:
        """
        Вычисление лексического разнообразия (1 - Jaccard similarity).
        
        Args:
            text1: Первый текст
            text2: Второй текст
            
        Returns:
            Лексическое разнообразие (от 0 до 1, выше = более разнообразно)
        """
        tokens1 = set(text1.lower().split())
        tokens2 = set(text2.lower().split())
        
        if not tokens1 or not tokens2:
            return 0.0
            
        intersection = len(tokens1 & tokens2)
        union = len(tokens1 | tokens2)
        
        jaccard = intersection / union if union > 0 else 0
        return 1 - jaccard
    
    def check_entailment(self, premise: str, hypothesis: str) -> Tuple[str, float]:
        """
        Проверка логического следования (entailment) между текстами.
        
        Args:
            premise: Исходное утверждение
            hypothesis: Проверяемое утверждение
            
        Returns:
            Кортеж (метка, уверенность)
        """
        result = self.nli_pipeline(
            hypothesis,
            candidate_labels=["entailment", "neutral", "contradiction"],
            hypothesis_template="This example is {}."
        )
        
        for label, score in zip(result["labels"], result["scores"]):
            if label == "entailment":
                return ("entailment", score)
        
        return (result["labels"][0], result["scores"][0])
    
    def validate(
        self,
        original: str,
        candidates: List[str],
        sim_min: float = 0.7,
        sim_max: float = 0.95,
        diversity_min: float = 0.1,
        use_nli: bool = True,
        nli_threshold: float = 0.5,
        check_sentence_type: bool = True
    ) -> List[Tuple[str, dict]]:
        """
        Валидация списка кандидатов-парафразов.
        
        Args:
            original: Исходный текст
            candidates: Список кандидатов
            sim_min: Минимальный порог сходства
            sim_max: Максимальный порог сходства (для отсеивания копий)
            diversity_min: Минимальное лексическое разнообразие
            use_nli: Использовать ли NLI проверку
            nli_threshold: Порог для NLI entailment
            check_sentence_type: Проверять ли сохранение типа предложения
            
        Returns:
            Список кортежей (парафраз, метрики) для прошедших валидацию
        """
        validated = []
        
        word_count = len(original.split())
        if word_count <= 6:
            effective_sim_min = max(0.6, sim_min - 0.1)
            effective_nli_threshold = max(0.3, nli_threshold - 0.15)
        else:
            effective_sim_min = sim_min
            effective_nli_threshold = nli_threshold
        
        for candidate in candidates:
            metrics = {}
            
            if check_sentence_type:
                if not preserve_sentence_type(original, candidate):
                    metrics["rejected"] = f"sentence type changed"
                    continue
            
            similarity = self.compute_similarity(original, candidate)
            metrics["similarity"] = similarity
            
            if similarity < effective_sim_min or similarity > sim_max:
                metrics["rejected"] = f"similarity {similarity:.3f} not in [{effective_sim_min}, {sim_max}]"
                continue
            
            diversity = self.compute_lexical_diversity(original, candidate)
            metrics["lexical_diversity"] = diversity
            
            if diversity < diversity_min:
                metrics["rejected"] = f"diversity {diversity:.3f} < {diversity_min}"
                continue
            
            if use_nli:
                label_fwd, score_fwd = self.check_entailment(original, candidate)
                label_bwd, score_bwd = self.check_entailment(candidate, original)
                
                metrics["nli_forward"] = {"label": label_fwd, "score": score_fwd}
                metrics["nli_backward"] = {"label": label_bwd, "score": score_bwd}
                
                if label_fwd != "entailment" or score_fwd < effective_nli_threshold:
                    metrics["rejected"] = f"NLI forward: {label_fwd} ({score_fwd:.3f})"
                    continue
                if label_bwd != "entailment" or score_bwd < effective_nli_threshold:
                    metrics["rejected"] = f"NLI backward: {label_bwd} ({score_bwd:.3f})"
                    continue
            
            metrics["accepted"] = True
            validated.append((candidate, metrics))
        
        target_sim = (effective_sim_min + sim_max) / 2
        validated.sort(key=lambda x: abs(x[1]["similarity"] - target_sim))
        
        return validated

### Инициализация валидатора

In [9]:
validator = ParaphraseValidator()

Загрузка модели эмбеддингов: sentence-transformers/all-MiniLM-L6-v2
Загрузка NLI модели: facebook/bart-large-mnli


Device set to use cpu


Модели валидатора загружены успешно!


### Тестирование отдельных метрик валидатора

In [10]:
original = "What's the weather like today?"
paraphrases_to_test = [
    "How is the weather today?",
    "What is today's weather forecast?",
    "Tell me about the weather.",
    "I like pizza.",  # Совсем другой смысл
]

print(f"Исходный текст: {original}")
print("=" * 60)
print("\nСемантическое сходство и лексическое разнообразие:")
print("-" * 60)

for p in paraphrases_to_test:
    sim = validator.compute_similarity(original, p)
    div = validator.compute_lexical_diversity(original, p)
    print(f"Similarity: {sim:.3f} | Diversity: {div:.3f} | {p}")

Исходный текст: What's the weather like today?

Семантическое сходство и лексическое разнообразие:
------------------------------------------------------------
Similarity: 0.911 | Diversity: 0.571 | How is the weather today?
Similarity: 0.744 | Diversity: 0.889 | What is today's weather forecast?
Similarity: 0.780 | Diversity: 0.889 | Tell me about the weather.
Similarity: 0.158 | Diversity: 0.857 | I like pizza.


### Тестирование NLI

In [11]:
original = "Please turn on the lights in the living room."
paraphrases_to_test = [
    "Switch on the living room lights.",
    "Turn off the lights.",  # Противоположный смысл
    "The lights are on.",  # Утверждение, не команда
]

print(f"Исходный текст: {original}")
print("=" * 60)
print("\nNLI проверка (entailment в обе стороны):")
print("-" * 60)

for p in paraphrases_to_test:
    label_fwd, score_fwd = validator.check_entailment(original, p)
    label_bwd, score_bwd = validator.check_entailment(p, original)
    print(f"\n{p}")
    print(f"  Forward:  {label_fwd} ({score_fwd:.3f})")
    print(f"  Backward: {label_bwd} ({score_bwd:.3f})")

Исходный текст: Please turn on the lights in the living room.

NLI проверка (entailment в обе стороны):
------------------------------------------------------------

Switch on the living room lights.
  Forward:  entailment (0.550)
  Backward: entailment (0.508)

Turn off the lights.
  Forward:  entailment (0.286)
  Backward: entailment (0.508)

The lights are on.
  Forward:  entailment (0.398)
  Backward: entailment (0.508)


## 5. Основной пайплайн

Объединённый пайплайн для генерации и валидации парафразов.

In [12]:
class ParaphrasePipeline:
    """
    Объединенный пайплайн для генерации и валидации парафразов.
    """
    
    def __init__(
        self,
        generator: Optional[ParaphraseGenerator] = None,
        validator: Optional[ParaphraseValidator] = None,
        device: Optional[str] = None
    ):
        """
        Инициализация пайплайна.
        
        Args:
            generator: Экземпляр генератора (создается по умолчанию если None)
            validator: Экземпляр валидатора (создается по умолчанию если None)
            device: Устройство для вычислений
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        
        self.generator = generator or ParaphraseGenerator(device=self.device)
        self.validator = validator or ParaphraseValidator(device=self.device)
    
    def run(
        self,
        text: str,
        num_paraphrases: int = 10,
        num_candidates: int = 15,
        strategy: str = "nucleus",
        validate: bool = True,
        use_nli: bool = True,
        sim_min: float = 0.7,
        sim_max: float = 0.95,
        verbose: bool = True
    ) -> List[str]:
        """
        Запуск полного пайплайна генерации парафразов.
        
        Args:
            text: Исходный текст
            num_paraphrases: Желаемое количество парафразов на выходе
            num_candidates: Количество кандидатов для генерации
            strategy: Стратегия генерации
            validate: Применять ли валидацию
            use_nli: Использовать ли NLI в валидации
            sim_min: Минимальный порог сходства
            sim_max: Максимальный порог сходства
            verbose: Выводить ли подробную информацию
            
        Returns:
            Список валидированных парафразов
        """
        if verbose:
            print(f"\n{'='*60}")
            print(f"Исходный текст: {text}")
            print(f"Тип предложения: {detect_sentence_type(text)}")
            print(f"{'='*60}")
        
        word_count = len(text.split())
        if word_count <= 6:
            effective_candidates = max(num_candidates, 20)
        else:
            effective_candidates = num_candidates
        
        if verbose:
            print(f"\nГенерация {effective_candidates} кандидатов (стратегия: {strategy})...")
        
        candidates = self.generator.generate(
            text,
            num_paraphrases=effective_candidates,
            strategy=strategy
        )
        
        if verbose:
            print(f"Сгенерировано {len(candidates)} уникальных кандидатов")
        
        if not validate:
            return candidates[:num_paraphrases]
        
        if verbose:
            print(f"\nВалидация кандидатов...")
        
        validated = self.validator.validate(
            text,
            candidates,
            sim_min=sim_min,
            sim_max=sim_max,
            use_nli=use_nli
        )
        
        if verbose:
            print(f"Прошло валидацию: {len(validated)} из {len(candidates)}")
            
            if validated:
                print(f"\nРезультаты:")
                for i, (paraphrase, metrics) in enumerate(validated[:num_paraphrases], 1):
                    sim = metrics.get("similarity", 0)
                    div = metrics.get("lexical_diversity", 0)
                    print(f"  {i}. {paraphrase}")
                    print(f"     [similarity: {sim:.3f}, diversity: {div:.3f}]")
        
        return [p for p, _ in validated[:num_paraphrases]]

### Создание пайплайна с уже загруженными моделями

In [13]:
pipe = ParaphrasePipeline(generator=generator, validator=validator)
print("Пайплайн готов к работе!")

Пайплайн готов к работе!


## 6. Демонстрация

Запустим полный пайплайн на примерах разных типов предложений.

### Пример 1: Длинное утверждение/совет

In [14]:
text = "After your workout, remember to focus on maintaining a good water balance."
paraphrases = pipe.run(text, num_paraphrases=5, num_candidates=15)


Исходный текст: After your workout, remember to focus on maintaining a good water balance.
Тип предложения: statement

Генерация 15 кандидатов (стратегия: nucleus)...
Сгенерировано 15 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 10 из 15

Результаты:
  1. Keep a suitable water intake after completing your exercise regimen.
     [similarity: 0.808, diversity: 0.778]
  2. Keep your water consumption level high after a workout.
     [similarity: 0.874, diversity: 0.765]
  3. Remember to maintain a healthy water balance after your exercise session.
     [similarity: 0.877, diversity: 0.647]
  4. Keep your water consumption levels within the healthy range after exercising.
     [similarity: 0.762, diversity: 0.850]
  5. Make sure to keep a good water intake after your workout.
     [similarity: 0.896, diversity: 0.647]


### Пример 2: Команда (включить свет)

In [15]:
text = "Please turn on the lights in the living room."
paraphrases = pipe.run(text, num_paraphrases=5, num_candidates=15)


Исходный текст: Please turn on the lights in the living room.
Тип предложения: command

Генерация 15 кандидатов (стратегия: nucleus)...
Сгенерировано 14 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 2 из 14

Результаты:
  1. Please activate the lights fixture in the living room.
     [similarity: 0.847, diversity: 0.400]
  2. It is my responsibility to light up the living room.
     [similarity: 0.710, diversity: 0.800]


### Пример 3: Команда (установить будильник)

In [16]:
text = "Set an alarm for 7 AM tomorrow."
paraphrases = pipe.run(text, num_paraphrases=5, num_candidates=15)


Исходный текст: Set an alarm for 7 AM tomorrow.
Тип предложения: command

Генерация 15 кандидатов (стратегия: nucleus)...
Сгенерировано 8 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 0 из 8


### Пример 4: Команда (напоминание)

In [17]:
text = "Remind me to call mom at 5 PM."
paraphrases = pipe.run(text, num_paraphrases=5, num_candidates=15)


Исходный текст: Remind me to call mom at 5 PM.
Тип предложения: command

Генерация 15 кандидатов (стратегия: nucleus)...
Сгенерировано 15 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 12 из 15

Результаты:
  1. Don't forget to contact mom at 5 PM.
     [similarity: 0.813, diversity: 0.545]
  2. At 5 PM, please make a call to my mother.
     [similarity: 0.838, diversity: 0.714]
  3. Please remind me to reach out to my mom at 5 PM.
     [similarity: 0.852, diversity: 0.417]
  4. At 5 PM, please reach out to mom and tell her.
     [similarity: 0.785, diversity: 0.733]
  5. Call mom at 5 PM
     [similarity: 0.867, diversity: 0.556]


### Пример 5: Вопрос о погоде

In [18]:
text = "What's the weather like today?"
paraphrases = pipe.run(text, num_paraphrases=5, num_candidates=15)


Исходный текст: What's the weather like today?
Тип предложения: question

Генерация 20 кандидатов (стратегия: nucleus)...
Сгенерировано 12 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 8 из 12

Результаты:
  1. What's the weather forecast for today?
     [similarity: 0.778, diversity: 0.429]
  2. Is there any news about the weather today?
     [similarity: 0.737, diversity: 0.700]
  3. Can you tell me what's happening in the weather today?
     [similarity: 0.830, diversity: 0.636]
  4. Can you tell me about the weather today?
     [similarity: 0.863, diversity: 0.700]
  5. What's the current weather like?
     [similarity: 0.895, diversity: 0.571]


### Пример 6: Вопрос о маршруте

In [19]:
text = "How do I get to the nearest coffee shop?"
paraphrases = pipe.run(text, num_paraphrases=5, num_candidates=15)


Исходный текст: How do I get to the nearest coffee shop?
Тип предложения: question

Генерация 15 кандидатов (стратегия: nucleus)...
Сгенерировано 13 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 11 из 13

Результаты:
  1. What is the best way to locate a nearby coffee shop?
     [similarity: 0.852, diversity: 0.750]
  2. Where is the closest coffee shop to my house?
     [similarity: 0.859, diversity: 0.800]
  3. What is the closest coffee shop to me?
     [similarity: 0.860, diversity: 0.786]
  4. What is the direction to get to the closest coffee shop?
     [similarity: 0.880, diversity: 0.615]
  5. How can I locate the closest coffee shop?
     [similarity: 0.882, diversity: 0.583]


## Эксперименты

Здесь вы можете попробовать свои примеры и настройки.

In [20]:
my_text = "Your text here"

num_paraphrases = 5      # Сколько парафразов вернуть
num_candidates = 15      # Сколько кандидатов сгенерировать
strategy = "nucleus"     # beam, diverse_beam, sampling, nucleus
use_nli = True           # Использовать NLI проверку
sim_min = 0.7            # Минимальное сходство
sim_max = 0.95           # Максимальное сходство

paraphrases = pipe.run(
    my_text,
    num_paraphrases=num_paraphrases,
    num_candidates=num_candidates,
    strategy=strategy,
    use_nli=use_nli,
    sim_min=sim_min,
    sim_max=sim_max,
    verbose=True
)


Исходный текст: Your text here
Тип предложения: statement

Генерация 20 кандидатов (стратегия: nucleus)...
Сгенерировано 19 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 0 из 19


### Генерация без валидации (быстрее)

In [21]:
text = "What's the weather like today?"
paraphrases = pipe.run(
    text,
    num_paraphrases=10,
    validate=False,
    verbose=True
)


Исходный текст: What's the weather like today?
Тип предложения: question

Генерация 20 кандидатов (стратегия: nucleus)...
Сгенерировано 14 уникальных кандидатов


### Генерация без NLI (компромисс скорость/качество)

In [22]:
text = "Please turn on the lights in the living room."
paraphrases = pipe.run(
    text,
    num_paraphrases=5,
    use_nli=False,
    verbose=True
)


Исходный текст: Please turn on the lights in the living room.
Тип предложения: command

Генерация 15 кандидатов (стратегия: nucleus)...
Сгенерировано 15 уникальных кандидатов

Валидация кандидатов...
Прошло валидацию: 10 из 15

Результаты:
  1. I implore you to switch on the lights in the living room.
     [similarity: 0.827, diversity: 0.538]
  2. We request that you switch on the lights in the living room.
     [similarity: 0.823, diversity: 0.538]
  3. Please activate the lights fixture in the living room.
     [similarity: 0.847, diversity: 0.400]
  4. Please use the lights in the living room.
     [similarity: 0.863, diversity: 0.333]
  5. It is your request that the lights in the living room be on.
     [similarity: 0.778, diversity: 0.750]
